* 1段階目で特徴を抽出
* 2段階目でコード生成

In [29]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI

import sys
sys.path.append('../..')

from src.data.moon import moon_data
from src.data.pseudodata import pseudo_data
from src.data.titanic import titanic_data

from src.utils.data_to_text import data_to_text

llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

In [2]:
df_train, df_test = moon_data(num_train=300, seed = 3655)
df_train.head()

train data:300
test data:700


,Feature_1,Feature_2,target
644,0.069,-0.037,1
609,0.848,0.507,0
548,0.343,-0.249,1
513,-0.578,0.914,0
994,1.488,-0.376,1


In [12]:
column_name = df_train.columns 
column_name = ','.join(column_name)
column_name

'Feature_1,Feature_2,target'

In [13]:
train_str = data_to_text(df_train)

In [25]:
prompt = """
Please tell us quantitatively what you can find out from the following data without using machine learning or other methods.
------------
{column_name_}
{train_str_}
""".format(
    column_name_ = column_name,
    train_str_ = train_str
    )

In [26]:
eda_prompt = llm_model(prompt)

In [27]:
print(eda_prompt)

From the given data, we can observe the following:

1. The data consists of three columns: Feature_1, Feature_2, and target. 

2. The target column consists of binary values, 0.0 and 1.0, which suggests that this is a binary classification problem.

3. The Feature_1 and Feature_2 columns consist of continuous numerical values, both positive and negative.

4. For the target value of 1.0, the values of Feature_1 are generally positive and the values of Feature_2 are generally negative.

5. For the target value of 0.0, the values of Feature_1 are generally negative and the values of Feature_2 are generally positive.

6. There are some exceptions to the above observations, indicating that the relationship between the features and the target is not strictly linear.

7. The range of values for Feature_1 is approximately from -1.1 to 2.0, and for Feature_2 is approximately from -0.5 to 1.1.

8. The data does not appear to contain any missing or null values.

Without using machine learning or 

In [36]:
prompt2 = """
What is the code for quantitatively obtaining the following information from the data?
We do not need code for visualization or general EDA.
------------
EDA result
{eda_prompt_}
------------
data
{column_name_}
{train_str_}
""".format(
    eda_prompt_ = eda_prompt,
    column_name_ = column_name,
    train_str_ = train_str
    )
#print(prompt2)

In [37]:
eda_code = llm_model(prompt2)
print(eda_code)

Here is the Python code using pandas library to obtain the quantitative information from the data:

```python
import pandas as pd

# Assuming the data is in a csv file named 'data.csv'
data = pd.read_csv('data.csv')

# Checking the shape of the data
print(f"The data has {data.shape[0]} rows and {data.shape[1]} columns.")

# Checking the unique values in the target column
print(f"The unique values in the target column are: {data['target'].unique()}")

# Checking the general statistics for Feature_1 and Feature_2 for target value 1.0
print("Statistics for Feature_1 and Feature_2 when target is 1.0:")
print(data[data['target'] == 1.0][['Feature_1', 'Feature_2']].describe())

# Checking the general statistics for Feature_1 and Feature_2 for target value 0.0
print("Statistics for Feature_1 and Feature_2 when target is 0.0:")
print(data[data['target'] == 0.0][['Feature_1', 'Feature_2']].describe())

# Checking the range of values for Feature_1 and Feature_2
print(f"The range of values for Fe

In [38]:
print(df_train[df_train['target'] == 1.0][['Feature_1', 'Feature_2']].describe())

        Feature_1   Feature_2
count  150.000000  150.000000
mean     1.034240   -0.134093
std      0.703573    0.316721
min     -0.073000   -0.565000
25%      0.361500   -0.412750
50%      1.064500   -0.222500
75%      1.690000    0.099750
max      2.047000    0.585000


In [39]:
print(df_train[df_train['target'] == 0.0][['Feature_1', 'Feature_2']].describe())


        Feature_1   Feature_2
count  150.000000  150.000000
mean     0.010913    0.638067
std      0.706599    0.311549
min     -1.092000   -0.037000
25%     -0.657500    0.375500
50%      0.091000    0.699000
75%      0.714250    0.918500
max      1.008000    1.112000


In [ ]:
prompt = """
Please create your code in compliance with all of the following conditions. Output should be code only. Do not enclose the output in ``python ``` or the like.
・The following data are for "target" of 0 and 1, respectively. Analyze these data and create a python code to predict the probability that the "target" of the unknown data is 1.
------------------
{eda_code_}
------------------
・Create a code that predicts a high probability value when "target" is 1 and a low probability value when "target" is 0 for the data given above.
・Each data type is as follows. If necessary, you can change the data type.
------------------
{data_type_}
------------------
・The column names, in order, are as follows {col_name_}
・Think and code the logic to predict probability values based on the data without using a existing machine learning model.
・Please make your predictions as accurate as possible.
・Predicting probability values as finely as possible increases overall accuracy.
・If {col_option_} is not blank, add it after 'df = x.copy()'.
・You do not need to provide examples.
・Create a code like the following.
------------------
import numpy as np
def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.


        # Do not change the code after this point.
        output.append(y)
    return np.array(output)
""".format(
            dataset_str_ = dataset_str,
            data_type_ = data_type,
            col_name_ = col_name,
            col_option_ = col_option
            )
